In [5]:
import glob

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import cv2

In [32]:
images = {
    int(path[17:-4]): np.load(path).reshape((720, 1280, 3)) for path in glob.glob("data/photo_focal_*.npy")
}
print(len(images))
print(images.keys())

17
dict_keys([700, 4000, 500, 4600, 2500, 2000, 1900, 6100, 2600, 4700, 5200, 1500, 3100, 999, 2690, 3400, 1000])


In [3]:
canny_images = [
    cv2.Canny(cv2.blur(img, (3, 3)), 10, 70, apertureSize=3) for img in images.values()
]
# px.imshow(np.array(canny_images), animation_frame=0, width=5_00).show()

In [10]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
arucoParams = cv2.aruco.DetectorParameters()
(corners, ids, rejected) = cv2.aruco.detectMarkers(
    images[4600], arucoDict, parameters=arucoParams
)
# px.imshow(cv2.aruco.drawDetectedMarkers(images[0], corners, ids))

In [38]:
# 14.3, 14.4 cm shape
marker_size_mm = 144


def guess_focal(img: np.array, dist_mm: int, size_mm: int = marker_size_mm):
    check = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)[0]
    if not check:
        print(f"Warning: distance {dist_mm} can't be identified")
        return None
    cs = check[0][0]
    pixel_size = np.mean(
        [
            np.sqrt(np.square(cs[0] - cs[3]).sum()),
            np.sqrt(np.square(cs[1] - cs[2]).sum()),
        ]
    )
    return pixel_size * dist_mm / size_mm


guess_focal(images[4700], dist_mm=4700)

1697.382214334276

In [42]:
focals = []
for dist, im in images.items():
    f = guess_focal(im, dist)
    if f is None:
        continue
    focals.append(f)
    print(f"{dist: >7} & {focals[-1]: >10.2f} \\\\")
avg_focal = np.mean(focals)
print(f"\nAverage: {avg_focal}")

   4000 &    1736.67 \\
   4600 &    1709.78 \\
   2500 &    1727.65 \\
   2000 &    1736.81 \\
   1900 &    1729.10 \\
   6100 &    1694.97 \\
   2600 &    1715.37 \\
   4700 &    1697.38 \\
   5200 &    1733.71 \\
   1500 &    1761.04 \\
   3100 &    1701.58 \\
    999 &    1741.43 \\
   2690 &    1728.20 \\
   3400 &    1723.77 \\
   1000 &    1781.36 \\

Average: 1727.9219916873508


In [41]:
print(focals)
np.std(focals)

[1736.6692225138347, 1709.7781817118328, 1727.6480462816025, 1736.8053860134548, 1729.1015836927627, 1694.9738290574815, 1715.3728273179795, 1697.382214334276, 1733.7093989054363, 1761.041800181071, 1701.584317949083, 1741.429578781128, 1728.204649289449, 1723.7728754679363, 1781.355963812934]


22.42151082535732

In [9]:
test_cam_mat = np.array(
    [
        [avg_focal, 0, 0],
        [0, avg_focal, 0],
        [0, 0, 1],
    ]
)

In [10]:
images[500]

array([[[ 77,  76,  92],
        [ 75,  74,  90],
        [ 79,  76,  92],
        ...,
        [140, 122, 129],
        [145, 127, 134],
        [160, 142, 149]],

       [[ 79,  78,  94],
        [ 78,  77,  93],
        [ 77,  74,  90],
        ...,
        [141, 123, 130],
        [157, 139, 146],
        [171, 153, 160]],

       [[ 73,  74,  88],
        [ 72,  73,  87],
        [ 72,  71,  87],
        ...,
        [146, 126, 133],
        [163, 143, 150],
        [173, 153, 160]],

       ...,

       [[ 99,  81,  90],
        [103,  85,  94],
        [102,  83,  96],
        ...,
        [ 47,  43,  59],
        [ 47,  44,  58],
        [ 47,  44,  58]],

       [[100,  80,  89],
        [104,  84,  93],
        [104,  83,  95],
        ...,
        [ 45,  41,  57],
        [ 46,  42,  56],
        [ 46,  42,  56]],

       [[100,  80,  89],
        [100,  80,  89],
        [101,  80,  92],
        ...,
        [ 46,  42,  58],
        [ 46,  42,  56],
        [ 46,  42,  56]]

In [11]:
_object_points = np.array(
    [
        [-marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, -marker_size_mm / 2, 0],
        [-marker_size_mm / 2, -marker_size_mm / 2, 0],
    ]
)


def estimate_pose(cn, cam_internal=test_cam_mat, obj_points=_object_points):
    ret, r_vec, t_vec = cv2.solvePnP(
        obj_points,
        imagePoints=cn,
        cameraMatrix=cam_internal,
        distCoeffs=np.zeros((4,)),
        flags=cv2.SOLVEPNP_IPPE_SQUARE,
    )
    assert ret, "Failed to solve PnP for pose estimation!"
    r_mat = np.empty((3, 3))
    cv2.Rodrigues(r_vec, dst=r_mat)
    return r_mat, t_vec


estimate_pose(corners[0][0])

(array([[ 0.99987173,  0.01598344, -0.00102674],
        [ 0.01598207, -0.99987139, -0.00133023],
        [-0.00104787,  0.00131365, -0.99999859]]),
 array([[2344.26555417],
        [1932.78990169],
        [5054.89266862]]))

In [2]:
cam_test = np.array([
    np.load(f"data/cam_rotate_{nth}.npy") for nth in range(20)
])
len(cam_test)

20

In [30]:
(list(images.values()))[0].shape

(1280, 720, 3)

In [ ]:
px.imshow(np.array(list(images.values())), animation_frame=0).show()